### Импортируем библиотеки

Увеличиваем ширину рабочей области на максимум

In [ ]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [ ]:
import pandas as pd
import os

### Функция для объединения файлов прогнозов

In [ ]:
#Функция объединения, содержащее только необходимые столбцы прогнозных шаблонов для загрузки в дашборд
def merging_files_dsh(dir_in, dir_out, file_name):
    #dir_in - расположение файлов прогнозов
    #dir_out - куда сохраняем итоговые файл
    #file_name - наименование итогового файла
    #cols_v = ['ФП.1','MIR-код','ПФМ','Программа бюджета','1КВ','2КВ','3КВ','4КВ'] #учитываемые столбцы в прогнозных файлах, ФП.1 т.к. pandas воспринимаем первый столбец как индексный, и атвоматом создается ФП.1 уже со значениями
    cols_v = ['ФП.1','MIR-код','ПФМ','Программа бюджета','Срез','1КВ','2КВ','3КВ','4КВ']
    df_ttl = pd.DataFrame() # пустой датафрейм для итоговой вствки
    for entry in os.scandir(dir_in): #перебираем все файлы в папке
        if entry.is_file() and entry.name.endswith('.xlsx'): #если файлы соответствуют условию, то учитываем
            print(entry.path)
            #читаем книгу
            df = pd.read_excel(open(entry.path,'rb'), sheet_name='Отчет',engine='openpyxl', header=1, index_col = None, usecols=cols_v).rename(columns={'ФП.1':'ФП'})  
            #удаляем все строки не относящиеся к факту (без аналитик и итоговые строки)
            df = df[(df['Срез']=='Факт 22/Прогноз') & (df['ФП'].str.contains('9_')==False)].reset_index(drop=True)
            #если итоговая таблица пустая, значит это первая итерация и сохраняем первый df, если второй то добавляем к нему доп. дф
            if df_ttl.empty:
                df_ttl = df
            else:
                df_ttl = pd.concat([df_ttl, df])
            #очищаем df
            df = None
    #разворачиваем колонки в строки
    df_ttl = df_ttl.melt(id_vars=['ФП','MIR-код','ПФМ','Программа бюджета'], value_vars=['1КВ', '2КВ','3КВ','4КВ'], var_name = 'quarter',value_name='amount')
    #оставляем в столбце квартала только цифру
    df_ttl['quarter'] = df_ttl['quarter'].str.extract('(\d+)')
    #удаляем из файла строки, которые пустые в сумме
    df_ttl = df_ttl[df_ttl['amount'] != 0].dropna(subset=['amount'])
    #сохраняем
    df_ttl.to_excel(dir_out + '\\' + file_name + '_dsh.xlsx', sheet_name = 'Отчет', encoding = 'windows-1251', index = False)
    #return df_ttl

In [ ]:
#Функция объединения, содержащее расширенный набор столбцов (из справочников) прогнозных шаблонов для анализа. Используется при необходимости
def merging_files_full(dir_in, dir_out, file_name):
    #dir_in - расположение файлов прогнозов
    #dir_out - куда сохраняем итоговые файл
    #file_name - наименование итогового файла
    #cols_v = ['ФП.1','MIR-код','ПФМ','Программа бюджета','1КВ','2КВ','3КВ','4КВ'] #учитываемые столбцы в прогнозных файлах, ФП.1 т.к. pandas воспринимаем первый столбец как индексный, и атвоматом создается ФП.1 уже со значениями
    cols_v = ['ФП.1','Наим ФП','MIR-код','ПФМ','Программа бюджета','Наим программы бюджета','Филиал','АУР/КВ','Срез','ГО/Сеть','1КВ','2КВ','3КВ','4КВ']
    df_ttl = pd.DataFrame() # пустой датафрейм для итоговой вствки
    for entry in os.scandir(dir_in): #перебираем все файлы в папке
        if entry.is_file() and entry.name.endswith('.xlsx'): #если файлы соответствуют условию, то учитываем
            print(entry.path)
            #читаем книгу
            df = pd.read_excel(open(entry.path,'rb'), sheet_name='Отчет',engine='openpyxl', header=1, index_col = None, usecols=cols_v).rename(columns={'ФП.1':'ФП'})  
            #удаляем все строки не относящиеся к факту (без аналитик и итоговые строки)
            df = df[(df['Срез'].str.contains('П-Б|П/Б')==False) & (df['ФП'].str.contains('9_')==False)].reset_index(drop=True)
            #если итоговая таблица пустая, значит это первая итерация и сохраняем первый df, если второй то добавляем к нему доп. дф
            if df_ttl.empty:
                df_ttl = df
            else:
                df_ttl = pd.concat([df_ttl, df])
            #очищаем df
            df = None
    #разворачиваем колонки в строки
    df_ttl = df_ttl.melt(id_vars=['ФП','Наим ФП','MIR-код','ПФМ','Программа бюджета','Наим программы бюджета','Филиал','АУР/КВ','Срез','ГО/Сеть'], value_vars=['1КВ', '2КВ','3КВ','4КВ'], var_name = 'quarter',value_name='amount')
    #оставляем в столбце квартала только цифру
    df_ttl['quarter'] = df_ttl['quarter'].str.extract('(\d+)')
    #удаляем из файла строки, которые пустые в сумме
    df_ttl = df_ttl[df_ttl['amount'] != 0].dropna(subset=['amount'])
    #сохраняем
    df_ttl.to_excel(dir_out + '\\' + file_name + '_full.xlsx', sheet_name = 'Отчет', encoding = 'windows-1251', index = False)
    #return df_ttl

### Запускаем функцию

Общая папка для вставки

In [ ]:
# папка для вставки
out_folder = r'\\*\Прогноз\2022'

Данные ГО

In [ ]:
# папка с файлами ГО
hq_folder = r'\\*\09.2022'
#наименование выходного файла
period_v = '4Q-2022'
file_v = 'Итоговый прогноз ГО за ' + period_v
#запуск функции
merging_files_dsh (hq_folder,out_folder,file_v)

Данные ЦАП

In [ ]:
# папка с файлами филиалов
branches_folder = r'\\*'
#наименование выходного файла
period_v = '4Q-2022'
file_v = 'Итоговый прогноз филиалов за ' + period_v
#запуск функции
merging_files_dsh (branches_folder,out_folder,file_v)